In [1]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import sklearn
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix,classification_report
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

In [7]:
#import file
Employee =pd.read_csv("Employee.csv")

In [ ]:
#แปลงType
Employee['employee_id'] = Employee['employee_id'].astype('category')
Employee['awards_won?'] = Employee['awards_won?'].astype('category')
Employee['is_promoted'] = Employee['is_promoted'].astype('category')

#เช็คจำนวนและTypeของData หลังแปลง Type แล้ว
Employee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48660 entries, 0 to 48659
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   employee_id           48660 non-null  category
 1   department            48660 non-null  object  
 2   region                48660 non-null  object  
 3   education             48660 non-null  object  
 4   gender                48660 non-null  object  
 5   recruitment_channel   48660 non-null  object  
 6   no_of_trainings       48660 non-null  int64   
 7   age                   48660 non-null  int64   
 8   previous_year_rating  48660 non-null  int64   
 9   length_of_service     48660 non-null  int64   
 10  awards_won?           48660 non-null  category
 11  avg_training_score    48660 non-null  int64   
 12  is_promoted           48660 non-null  category
dtypes: category(3), int64(5), object(5)
memory usage: 5.4+ MB


In [ ]:
#สำหรับ SVM
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# แยกคอลัมน์เชิงปริมาณกับเชิงกลุ่ม
quant_cols = Employee.select_dtypes(include=['int64', 'float64']).columns
cat_cols = Employee.select_dtypes(include=['object', 'category']).columns
cat_cols = cat_cols.drop('employee_id','is_promoted')
target = Employee['is_promoted']

# Standardize ข้อมูลเชิงปริมาณ
scaler = StandardScaler()
quant_scaled = pd.DataFrame(scaler.fit_transform(Employee[quant_cols]), columns=quant_cols)

# 2. One-hot encoding ข้อมูลเชิงกลุ่ม
encoder = OneHotEncoder(sparse_output=False)  # drop='first' เพื่อลด multicollinearity
cat_encoded = pd.DataFrame(encoder.fit_transform(Employee[cat_cols]),
                           columns=encoder.get_feature_names_out(cat_cols))

quant_cols = pd.DataFrame(quant_cols)
# 3. รวมข้อมูลกลับ
df_standard = pd.concat([quant_scaled.reset_index(drop=True), cat_encoded.reset_index(drop=True), target.reset_index(drop=True)], axis=1)
df_standard = df_standard.drop(columns=['is_promoted_0','is_promoted_1'])



In [ ]:
#สำหรับข้อมูลปกติ

# แยกคอลัมน์เชิงปริมาณกับเชิงกลุ่ม
quant_cols = Employee.select_dtypes(include=['int64', 'float64']).columns
cat_cols = Employee.select_dtypes(include=['object', 'category']).columns

# ลบคอลัมน์ที่ไม่ต้องการ one-hot encode
cat_cols = cat_cols.drop(['employee_id', 'is_promoted'])

# แยก target variable
target = Employee['is_promoted']

# One-hot encoding ข้อมูลเชิงกลุ่ม
encoder = OneHotEncoder(sparse_output=False)
cat_encoded = pd.DataFrame(encoder.fit_transform(Employee[cat_cols]),
                           columns=encoder.get_feature_names_out(cat_cols),
                           index=Employee.index)

# ดึงข้อมูลเชิงปริมาณ (DataFrame จริง)
quant_data = Employee[quant_cols]

# รวมข้อมูลกลับ
df_final = pd.concat([quant_data, cat_encoded,target], axis=1)
#df_final = df_final.drop(columns=['is_promoted_0','is_promoted_1'])


# การนำเข้าข้อมูล

In [ ]:
#
Employee.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5,8,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5,4,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3,7,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1,10,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3,2,0,73,0


In [ ]:
#เช็คจำนวนและ Type ของ Data
Employee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48660 entries, 0 to 48659
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   employee_id           48660 non-null  category
 1   department            48660 non-null  object  
 2   region                48660 non-null  object  
 3   education             48660 non-null  object  
 4   gender                48660 non-null  object  
 5   recruitment_channel   48660 non-null  object  
 6   no_of_trainings       48660 non-null  int64   
 7   age                   48660 non-null  int64   
 8   previous_year_rating  48660 non-null  int64   
 9   length_of_service     48660 non-null  int64   
 10  awards_won?           48660 non-null  category
 11  avg_training_score    48660 non-null  int64   
 12  is_promoted           48660 non-null  category
dtypes: category(3), int64(5), object(5)
memory usage: 5.4+ MB


In [ ]:
Employee.isnull().sum()

,0
employee_id,0
department,0
region,0
education,0
gender,0
recruitment_channel,0
no_of_trainings,0
age,0
previous_year_rating,0
length_of_service,0


In [ ]:
# ทำการเคลียร์ค่าว่างทั้งหมด
Employee = Employee.dropna()

In [ ]:
# Check Missing Value before clean missing value
Employee.isnull().sum()

,0
employee_id,0
department,0
region,0
education,0
gender,0
recruitment_channel,0
no_of_trainings,0
age,0
previous_year_rating,0
length_of_service,0


In [ ]:
Employee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48660 entries, 0 to 48659
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   employee_id           48660 non-null  category
 1   department            48660 non-null  object  
 2   region                48660 non-null  object  
 3   education             48660 non-null  object  
 4   gender                48660 non-null  object  
 5   recruitment_channel   48660 non-null  object  
 6   no_of_trainings       48660 non-null  int64   
 7   age                   48660 non-null  int64   
 8   previous_year_rating  48660 non-null  int64   
 9   length_of_service     48660 non-null  int64   
 10  awards_won?           48660 non-null  category
 11  avg_training_score    48660 non-null  int64   
 12  is_promoted           48660 non-null  category
dtypes: category(3), int64(5), object(5)
memory usage: 5.4+ MB


In [ ]:
Employee.duplicated().sum()

np.int64(0)

In [ ]:
#หาค่าสถิติเบื้องต้น
Employee.describe()

,no_of_trainings,age,previous_year_rating,length_of_service,avg_training_score
count,48660.000000,48660.000000,48660.000000,48660.00000,48660.000000
mean,1.251993,35.589437,3.337526,6.31157,63.603309
std,0.604994,7.534571,1.257922,4.20476,13.273502
min,1.000000,20.000000,1.000000,1.00000,39.000000
25%,1.000000,30.000000,3.000000,3.00000,51.000000
50%,1.000000,34.000000,3.000000,5.00000,60.000000
75%,1.000000,39.000000,4.000000,8.00000,76.000000
max,10.000000,60.000000,5.000000,37.00000,99.000000


In [ ]:
Employee

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5,8,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5,4,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3,7,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1,10,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3,2,0,73,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48655,6915,Sales & Marketing,region_14,Bachelor's,m,other,2,31,1,2,0,49,0
48656,3030,Technology,region_14,Bachelor's,m,sourcing,1,48,3,17,0,78,0
48657,74592,Operations,region_27,Master's & above,f,other,1,37,2,6,0,56,0
48658,13918,Analytics,region_1,Bachelor's,m,other,1,27,5,3,0,79,0


In [ ]:
# คำนวณเมทริกซ์ความสัมพันธ์
corelation_matrix = Employee.corr().round(2)

# เลือกคอลัมน์ที่เกี่ยวข้องกับตัวแปรเป้าหมาย (Y)
target_correlation = corelation_matrix[['is_promoted']].sort_values(by='is_promoted', ascending=False)

# เลือกเฉพาะคอลัมน์ที่มีค่าความสัมพันธ์สูงสุด 10 อันดับแรก
top_10_correlation = target_correlation.head(31)

# วาด heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(top_10_correlation, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation with is_promoted (Y)')
plt.show()

# Encododer และ เริ่มเตรียมข้อมูลก่อนเข้าModel

In [ ]:
# แบ่ง X และ Y
x = df_final.drop(columns=['is_promoted'])
y = df_final['is_promoted']

In [ ]:
# Split data into training and testing sets

# train 70 test 30
x_train70, x_test30, y_train70, y_test30 = train_test_split(x, y, test_size=0.3, random_state=42)

# train 80 test 20
x_train80, x_test20, y_train80, y_test20 = train_test_split(x,y, test_size=0.2, random_state=42)


In [ ]:
# Handle data imbalance
smote = SMOTE(random_state=42, k_neighbors=5)
# 70 : 30
x_train_smote70, y_train_smote70 = smote.fit_resample(x_train70, y_train70)
x_test_smote70, y_test_smote70 = smote.fit_resample(x_test30, y_test30)
# 80 : 20
x_train_smote80, y_train_smote80 = smote.fit_resample(x_train80, y_train80)
x_test_smote80, y_test_smote80 = smote.fit_resample(x_test20, y_test20)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# จำนวนฟีเจอร์ที่ต้องการเลือก
n_features_to_select = 5

# สร้างตัวประมาณค่าพื้นฐาน
estimator = LogisticRegression()

# สำหรับชุดข้อมูล 70%
selector70 = RFE(estimator, n_features_to_select=n_features_to_select)
x_train_selected70 = selector70.fit_transform(x_train70, y_train70)
x_test_selected30 = selector70.transform(x_test30)

# สำหรับชุดข้อมูล 80%
selector80 = RFE(estimator, n_features_to_select=n_features_to_select)
x_train_selected80 = selector80.fit_transform(x_train80, y_train80)
x_test_selected20 = selector80.transform(x_test20)

# สำหรับชุดข้อมูล SMOTE 70%
selector_smote70 = RFE(estimator, n_features_to_select=n_features_to_select)
x_train_smoteselected70 = selector_smote70.fit_transform(x_train_smote70, y_train_smote70)
x_test_smoteselected30 = selector_smote70.transform(x_test_smote70)

# สำหรับชุดข้อมูล SMOTE 80%
selector_smote80 = RFE(estimator, n_features_to_select=n_features_to_select)
x_train_smoteselected80 = selector_smote80.fit_transform(x_train_smote80, y_train_smote80)
x_test_smoteselected20 = selector_smote80.transform(x_test_smote80)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

# ข้อมูลสำหรับ SVM ที่ปรับค่ามาตรฐาน

In [ ]:
stand_x = df_standard.drop(columns=['is_promoted'])
stand_y = df_standard['is_promoted']

In [ ]:
# Split data into training and testing sets

# train 70 test 30
x_train70Stand, x_test30Stand, y_train70Stand, y_test30Stand = train_test_split(stand_x, stand_y, test_size=0.3, random_state=42)

# train 80 test 20
x_train80Stand, x_test20Stand, y_train80Stand, y_test20Stand = train_test_split(stand_x, stand_y, test_size=0.2, random_state=42)

In [ ]:
# Handle data imbalance
smote = SMOTE(random_state=42, k_neighbors=5)
# 70 : 30
x_train_smote70Stand, y_train_smote70Stand = smote.fit_resample(x_train70Stand, y_train70Stand)
x_test_smote30Stand, y_test_smote30Stand = smote.fit_resample(x_test30Stand, y_test30Stand)
# 80 : 20
x_train_smote80Stand, y_train_smote80Stand = smote.fit_resample(x_train80Stand, y_train80Stand)
x_test_smote20Stand, y_test_smote20Stand = smote.fit_resample(x_test20Stand, y_test20Stand)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# จำนวนฟีเจอร์ที่ต้องการเลือก
n_features_to_select = 5
# สร้างตัวประมาณค่าพื้นฐาน
estimator = LogisticRegression()

# สำหรับชุดข้อมูล 70%
selector70 = RFE(estimator, n_features_to_select=n_features_to_select)
x_train_selected70Stand = selector70.fit_transform(x_train70Stand, y_train70Stand)
x_test_selected30Stand = selector70.transform(x_test30Stand)

# สำหรับชุดข้อมูล 80%
selector80 = RFE(estimator, n_features_to_select=n_features_to_select)
x_train_selected80Stand = selector80.fit_transform(x_train80Stand, y_train80Stand)
x_test_selected20Stand = selector80.transform(x_test20Stand)

# สำหรับชุดข้อมูล SMOTE 70%
selector_smote70 = RFE(estimator, n_features_to_select=n_features_to_select)
x_train_smoteselected70Stand = selector_smote70.fit_transform(x_train_smote70Stand, y_train_smote70Stand)
x_test_smoteselected30Stand = selector_smote70.transform(x_test_smote30Stand)

# สำหรับชุดข้อมูล SMOTE 80%
selector_smote80 = RFE(estimator, n_features_to_select=n_features_to_select)
x_train_smoteselected80Stand = selector_smote80.fit_transform(x_train_smote80Stand, y_train_smote80Stand)
x_test_smoteselected20Stand = selector_smote80.transform(x_test_smote20Stand)

# Support Vector Machine without SMOTE and Feature Selection

In [ ]:
# ใช้ Cross Validation เพื่อปรับจูนค่าพารามิเตอร์  ของโมเดล Machine Learning เพื่อหาค่าที่เหมาะสมที่สุดในการเพิ่มประสิทธิภาพของโมเดล

# Support Vector Machine without SMOTE and Feature Selection   70 :30
param_grid = {
    'C': [0.01,0.1,1,10,100],
    'kernel': [ 'rbf'],
    'gamma': [0.1,1,10,100]
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(x_train70Stand, y_train70Stand)

print("Best Parameters:", grid.best_params_)
print("Best Score", grid.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=  14.2s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=  12.3s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=  15.0s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=  14.0s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=  13.9s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=  50.9s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=  50.4s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=  53.3s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=  49.7s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=  55.8s
[CV] END .......................C=0.01, gamma=10, kernel=rbf; total time= 1.3min
[CV] END .......................C=0.01, gamma=1

In [ ]:
# Train an SVM classifier
svm_model = SVC(kernel='rbf', C=10,gamma=0.1, random_state=42)
svm_model.fit(x_train70Stand, y_train70Stand)

# Make predictions
y_pred30 = svm_model.predict(x_test30Stand)

# แสดง Classification Report
accuracy = accuracy_score(y_test30Stand, y_pred30)
precision = precision_score(y_test30Stand, y_pred30)
recall = recall_score(y_test30Stand, y_pred30)
f1 = f1_score(y_test30Stand, y_pred30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.9337
Precision: 0.9606
Recall: 0.2775
F1 Score: 0.4306


In [ ]:
# Support Vector Machine without SMOTE and Feature Selection  80:20
param_grid = {
    'C': [0.01,0.1,1,10,100],
    'kernel': ['rbf'],
    'gamma': [0.1,1,10,100]
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(x_train80Stand, y_train80Stand)

print("Best Parameters:", grid.best_params_)
print("Best Score", grid.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=  15.4s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=  15.8s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=  16.1s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=  16.0s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=  15.4s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=  58.0s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 1.2min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 1.2min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=  56.3s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 1.2min
[CV] END .......................C=0.01, gamma=10, kernel=rbf; total time= 1.5min
[CV] END .......................C=0.01, gamma=1

In [ ]:
# Train an SVM classifier 80 : 20
svm_model = SVC(kernel='rbf', C=10,gamma=0.1, random_state=42)
svm_model.fit(x_train80Stand, y_train80Stand)

# Make predictions
y_pred20 = svm_model.predict(x_test20Stand)

# แสดง Classification Report  80 : 20
accuracy = accuracy_score(y_test20Stand, y_pred20)
precision = precision_score(y_test20Stand, y_pred20)
recall = recall_score(y_test20Stand, y_pred20)
f1 = f1_score(y_test20Stand, y_pred20)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.9354
Precision: 0.9508
Recall: 0.2895
F1 Score: 0.4439


# Support Vector Machine with SMOTE

In [ ]:
# Support Vector Machine with SMOTE  70 :30
param_grid = {
    'C': [0.01,0.1,1,10,100],
    'kernel': ['rbf'],
    'gamma': [0.1,1,10,100]
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(x_train_smote70Stand, y_train_smote70Stand)

print("Best Parameters:", grid.best_params_)
print("Best Score", grid.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 3.0min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 3.0min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 3.1min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 3.1min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 3.1min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 4.0min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 4.1min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 4.1min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 4.1min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 4.1min
[CV] END .......................C=0.01, gamma=10, kernel=rbf; total time= 3.1min
[CV] END .......................C=0.01, gamma=1

In [ ]:
# Train an SVM  With SMOTE  70 : 30
svm_model = SVC(kernel='rbf', C=10,gamma=1, random_state=42)
svm_model.fit(x_train_smote70Stand, y_train_smote70Stand)

# Make predictions
y_pred_smote30 = svm_model.predict(y_test_smote30Stand)

# แสดง Classification Report
accuracy = accuracy_score(y_test_smote30Stand, y_pred_smote30)
precision = precision_score(y_test_smote30Stand, y_pred_smote30)
recall = recall_score(y_test_smote30Stand, y_pred_smote30)
f1 = f1_score(y_test_smote30Stand, y_pred_smote30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

In [ ]:
# Support Vector Machine with SMOTE  80:20
param_grid = {
    'C': [0.01,0.1,1,10,100],
    'kernel': ['rbf'],
    'gamma': [0.1,1,10,100]
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(x_train_smote80Stand, y_train_smote80Stand)

print("Best Parameters:", grid.best_params_)
print("Best Score", grid.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 4.3min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 4.9min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 4.5min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 4.2min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 4.3min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 5.9min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 5.6min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 5.5min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 5.6min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 5.7min
[CV] END .......................C=0.01, gamma=10, kernel=rbf; total time= 5.0min
[CV] END .......................C=0.01, gamma=1

In [ ]:
# Train an SVM  With SMOTE  80 : 20
svm_model = SVC(kernel='rbf', C=10,gamma=1, random_state=42)
svm_model.fit(x_train_smote80Stand, y_train_smote80Stand)

# Make predictions
y_pred_smote20 = svm_model.predict(x_test_smote20Stand)

# แสดง Classification Report
accuracy = accuracy_score(y_test_smote20Stand, y_pred_smote20)
precision = precision_score(y_test_smote20Stand, y_pred_smote20)
recall = recall_score(y_test_smote20Stand, y_pred_smote20)
f1 = f1_score(y_test_smote20Stand, y_pred_smote20)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))


# Support Vector Machine with Feature Selection

In [ ]:
# Support Vector Machine with Feature Selection 70 : 30
param_grid = {
    'C': [0.01,0.1,1,10,100],
    'kernel': ['rbf'],
    'gamma': [0.1,1,10,100]
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(x_train_selected70Stand,y_train70Stand)

print("Best Parameters:", grid.best_params_)
print("Best Score", grid.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=   7.3s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=   7.1s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=   7.0s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=   6.7s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=   7.3s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=   7.3s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=   7.3s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=   7.3s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=   7.3s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=   7.5s
[CV] END .......................C=0.01, gamma=10, kernel=rbf; total time=   6.0s
[CV] END .......................C=0.01, gamma=1

In [ ]:
# Train an SVM classifier
svm_model = SVC(kernel='rbf', C=1,gamma=10, random_state=42)
svm_model.fit(x_train_selected70Stand,y_train70Stand)

# Make predictions
y_pred30 = svm_model.predict(x_test_selected30Stand)

# แสดง Classification Report
accuracy = accuracy_score(y_test30Stand, y_pred30)
precision = precision_score(y_test30Stand, y_pred30)
recall = recall_score(y_test30Stand, y_pred30)
f1 = f1_score(y_test30Stand, y_pred30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.9312
Precision: 0.9758
Recall: 0.2449
F1 Score: 0.3915


In [ ]:
# Support Vector Machine with Feature Selection 80 : 20
param_grid = {
    'C': [0.01,0.1,1,10,100],
    'kernel': ['rbf'],
    'gamma': [0.1,1,10,100]
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(x_train_selected80Stand, y_train80Stand)

print("Best Parameters:", grid.best_params_)
print("Best Score", grid.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=   8.8s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=   9.0s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=   8.7s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=   8.4s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=   8.8s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=   9.3s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=   9.5s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=   9.6s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=   9.3s
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time=   9.5s
[CV] END .......................C=0.01, gamma=10, kernel=rbf; total time=   8.5s
[CV] END .......................C=0.01, gamma=1

In [ ]:
# Train an SVM classifier 80 : 20
svm_model = SVC(kernel='rbf', C=1,gamma=10, random_state=42)
svm_model.fit(x_train_selected80Stand, y_train80Stand)

# Make predictions
y_pred20 = svm_model.predict(x_test_selected20Stand)

# แสดง Classification Report
accuracy = accuracy_score(y_test20Stand, y_pred20)
precision = precision_score(y_test20Stand, y_pred20)
recall = recall_score(y_test20Stand, y_pred20)
f1 = f1_score(y_test20Stand, y_pred20)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.9342
Precision: 0.9830
Recall: 0.2664
F1 Score: 0.4192


# Support Vector Machine with SMOTE and Feature Selection


In [ ]:
# Support Vector Machine with SMOTE And  Feature Selection 70 : 30
param_grid = {
    'C': [0.01,0.1,1,10,100],
    'kernel': ['rbf'],
    'gamma': [0.1,1,10,100]
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(x_train_smoteselected70Stand, y_train_smote70Stand)

print("Best Parameters:", grid.best_params_)
print("Best Score", grid.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 2.2min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 2.1min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 2.1min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 2.0min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 2.0min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 2.0min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 2.0min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 2.0min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 1.9min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 2.0min
[CV] END .......................C=0.01, gamma=10, kernel=rbf; total time= 1.9min
[CV] END .......................C=0.01, gamma=1

In [ ]:
# Train an SVM classifier
svm_model = SVC(kernel='rbf', C=100,gamma=100, random_state=42)
svm_model.fit(x_train_smoteselected70Stand, y_train_smote70Stand)
# Make predictions
y_pred30 = svm_model.predict(x_test_smoteselected30Stand)

# แสดง Classification Report
accuracy = accuracy_score(y_test_smote30Stand, y_pred30)
precision = precision_score(y_test_smote30Stand, y_pred30)
recall = recall_score(y_test_smote30Stand, y_pred30)
f1 = f1_score(y_test_smote30Stand, y_pred30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.6818
Precision: 0.7430
Recall: 0.5558
F1 Score: 0.6359


In [ ]:
# Support Vector Machine with SMOTE And  Feature Selection 80 : 20
param_grid = {
    'C': [0.01,0.1,1,10,100],
    'kernel': ['rbf'],
    'gamma': [0.1,1,10,100]
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(x_train_smoteselected80Stand, y_train_smote80Stand)

print("Best Parameters:", grid.best_params_)
print("Best Score", grid.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 1.8min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 1.8min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 1.8min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 1.8min
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time= 1.8min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 1.7min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 1.7min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 1.7min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 1.7min
[CV] END ........................C=0.01, gamma=1, kernel=rbf; total time= 1.7min
[CV] END .......................C=0.01, gamma=10, kernel=rbf; total time= 1.6min
[CV] END .......................C=0.01, gamma=1

In [ ]:
# Train an SVM classifier
svm_model = SVC(kernel='rbf', C=100,gamma=100, random_state=42)
svm_model.fit(x_train_smoteselected80Stand, y_train_smote80Stand)

# Make predictions
y_pred20 = svm_model.predict(x_test_smoteselected20Stand)

# แสดง Classification Report
accuracy = accuracy_score(y_test_smote20Stand, y_pred20)
precision = precision_score(y_test_smote20Stand, y_pred20)
recall = recall_score(y_test_smote20Stand, y_pred20)
f1 = f1_score(y_test_smote20Stand, y_pred20)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.5959
Precision: 0.6995
Recall: 0.3362
F1 Score: 0.4541


# Decistion Tree without SMOTE and Feature Selection

In [ ]:
# ใช้ Cross Validation เพื่อปรับจูนค่าพารามิเตอร์  ของโมเดล Machine Learning เพื่อหาค่าที่เหมาะสมที่สุดในการเพิ่มประสิทธิภาพของโมเดล Decistion Tree

# Decistion Tree without SMOTE and Feature Selection   70 :30

# Define the Decision Tree model
dt_classifier = DecisionTreeClassifier(random_state=42)

# Define the parameter grid for tuning
param_grid = {
    'max_depth': np.arange(2,11),         # ความลึกสูงสุดของต้นไม้
    'min_samples_split': np.arange(2,11),       # ขนาดขั้นต่ำของ sample ก่อนที่จะแบ่ง node
    'min_samples_leaf': np.arange(1,11), # ขนาดขั้นต่ำของ sample ใน leaf node
    'max_leaf_nodes' : [5,10,15,20]
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=dt_classifier,
                           param_grid=param_grid,
                           cv=5,                # 5-fold cross-validation
                           scoring='accuracy',  # ใช้คะแนนความแม่นยำ
                           n_jobs=-1)          # ใช้ทุก CPU core

# Fit the model to find the best parameters

grid_search.fit(x_train70, y_train70)

# Display the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Use the best estimator for predictions
best_model = grid_search.best_estimator_
print(best_model)

In [ ]:
# Train Decistion tree 70 :30
dt_model = DecisionTreeClassifier(
    max_depth=8,                # ความลึกสูงสุดของต้นไม้
    max_leaf_nodes=20,           # จำนวน leaf nodes สูงสุด
    min_samples_leaf=10,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=2,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (70%)
dt_model.fit(x_train70, y_train70)

# Make predictions on the test data (30%)
y_pred30 = dt_model.predict(x_test30)

# แสดง Classification Report
accuracy = accuracy_score(y_test30, y_pred30)
precision = precision_score(y_test30, y_pred30)
recall = recall_score(y_test30, y_pred30)
f1 = f1_score(y_test30, y_pred30)
print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.9275
Precision: 0.9221
Recall: 0.2153
F1 Score: 0.3491


In [ ]:
# Decistion Tree without SMOTE and Feature Selection   80 :20

# Define the Decision Tree model
dt_classifier = DecisionTreeClassifier(random_state=42)

# Define the parameter grid for tuning
param_grid = {
    'max_depth': np.arange(2,11),         # ความลึกสูงสุดของต้นไม้
    'min_samples_split': np.arange(2,11),       # ขนาดขั้นต่ำของ sample ก่อนที่จะแบ่ง node
    'min_samples_leaf': np.arange(1,11), # ขนาดขั้นต่ำของ sample ใน leaf node
    'max_leaf_nodes' : [5,10,15,20]
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=dt_classifier,
                           param_grid=param_grid,
                           cv=5,                # 5-fold cross-validation
                           scoring='accuracy',  # ใช้คะแนนความแม่นยำ
                           n_jobs=-1)          # ใช้ทุก CPU core

# Fit the model to find the best parameters

grid_search.fit(x_train80, y_train80)

# Display the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Use the best estimator for predictions
best_model = grid_search.best_estimator_
print(best_model)

Best Parameters: {'max_depth': 8, 'max_leaf_nodes': 20, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Cross-Validation Score: 0.929921897594302
DecisionTreeClassifier(max_depth=8, max_leaf_nodes=20, random_state=42)


In [ ]:
# Train Decistion tree 80 : 20
dt_model = DecisionTreeClassifier(
    max_depth=8,                # ความลึกสูงสุดของต้นไม้
    max_leaf_nodes=20,           # จำนวน leaf nodes สูงสุด
    min_samples_leaf=1,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=2,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (80%)
dt_model.fit(x_train80, y_train80)

# Make predictions on the test data (20%)
y_pred20 = dt_model.predict(x_test20)

# แสดง Classification Report
accuracy = accuracy_score(y_test20, y_pred20)
precision = precision_score(y_test20, y_pred20)
recall = recall_score(y_test20, y_pred20)
f1 = f1_score(y_test20, y_pred20)
print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.9263
Precision: 0.9121
Recall: 0.1915
F1 Score: 0.3165


# Decistion Tree with SMOTE

In [ ]:
# Decistion Tree with SMOTE 70 : 30

# Define the Decision Tree model
dt_classifier = DecisionTreeClassifier(random_state=42)

# Define the parameter grid for tuning
param_grid = {
    'max_depth': np.arange(2,11),         # ความลึกสูงสุดของต้นไม้
    'min_samples_split': np.arange(2,11),       # ขนาดขั้นต่ำของ sample ก่อนที่จะแบ่ง node
    'min_samples_leaf': np.arange(1,11), # ขนาดขั้นต่ำของ sample ใน leaf node
    'max_leaf_nodes' : [5,10,15,20]
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=dt_classifier,
                           param_grid=param_grid,
                           cv=5,                # 5-fold cross-validation
                           scoring='accuracy',  # ใช้คะแนนความแม่นยำ
                           n_jobs=-1)          # ใช้ทุก CPU core

# Fit the model to find the best parameters
grid_search.fit(x_train_smote70, y_train_smote70)

# Display the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Use the best estimator for predictions
best_model = grid_search.best_estimator_
print(best_model)

Best Parameters: {'max_depth': np.int64(10), 'max_leaf_nodes': 20, 'min_samples_leaf': np.int64(1), 'min_samples_split': np.int64(2)}
Best Cross-Validation Score: 0.8152435964280658
DecisionTreeClassifier(max_depth=np.int64(10), max_leaf_nodes=20,
                       min_samples_leaf=np.int64(1),
                       min_samples_split=np.int64(2), random_state=42)


In [ ]:
# Train Decistion Tree with SMOTE 70 : 30
dt_model = DecisionTreeClassifier(
    max_depth=10,                # ความลึกสูงสุดของต้นไม้
    max_leaf_nodes=20,           # จำนวน leaf nodes สูงสุด
    min_samples_leaf=1,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=2,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (70%)
dt_model.fit(x_train_smote70, y_train_smote70)

# Make predictions on the test data (30%)
y_pred30 = dt_model.predict(x_test_smote70)

# แสดง Classification Report
accuracy = accuracy_score(y_test_smote70, y_pred30)
precision = precision_score(y_test_smote70, y_pred30)
recall = recall_score(y_test_smote70, y_pred30)
f1 = f1_score(y_test_smote70, y_pred30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.8302
Precision: 0.9741
Recall: 0.6785
F1 Score: 0.7999


In [ ]:
# Decistion Tree with SMOTE 80 : 20

# Define the Decision Tree model
dt_classifier = DecisionTreeClassifier(random_state=42)

# Define the parameter grid for tuning
param_grid = {
    'max_depth': np.arange(2,11),         # ความลึกสูงสุดของต้นไม้
    'min_samples_split': np.arange(2,11),       # ขนาดขั้นต่ำของ sample ก่อนที่จะแบ่ง node
    'min_samples_leaf': np.arange(1,11), # ขนาดขั้นต่ำของ sample ใน leaf node
    'max_leaf_nodes' : [5,10,15,20]
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=dt_classifier,
                           param_grid=param_grid,
                           cv=5,                # 5-fold cross-validation
                           scoring='accuracy',  # ใช้คะแนนความแม่นยำ
                           n_jobs=-1)          # ใช้ทุก CPU core

# Fit the model to find the best parameters

grid_search.fit(x_train_smote80, y_train_smote80)

# Display the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Use the best estimator for predictions
best_model = grid_search.best_estimator_
print(best_model)

Best Parameters: {'max_depth': np.int64(10), 'max_leaf_nodes': 20, 'min_samples_leaf': np.int64(1), 'min_samples_split': np.int64(2)}
Best Cross-Validation Score: 0.8122219239161346
DecisionTreeClassifier(max_depth=np.int64(10), max_leaf_nodes=20,
                       min_samples_leaf=np.int64(1),
                       min_samples_split=np.int64(2), random_state=42)


In [ ]:
# Train Decistion Tree with SMOTE 80 : 20
dt_model = DecisionTreeClassifier(
    max_depth=10,                # ความลึกสูงสุดของต้นไม้
    max_leaf_nodes=20,           # จำนวน leaf nodes สูงสุด
    min_samples_leaf=1,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=2,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (80%)
dt_model.fit(x_train_smote80, y_train_smote80)

# Make predictions on the test data (20%)
y_pred20 = dt_model.predict(x_test_smote80)

# แสดง Classification Report
accuracy = accuracy_score(y_test_smote80, y_pred20)
precision = precision_score(y_test_smote80, y_pred20)
recall = recall_score(y_test_smote80, y_pred20)
f1 = f1_score(y_test_smote80, y_pred20)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.8342
Precision: 0.9735
Recall: 0.6872
F1 Score: 0.8057


# Decision Tree with Feature Selection


In [ ]:
# Decision Tree with Feature Selection 70 : 30

# Define the Decision Tree model
dt_classifier = DecisionTreeClassifier(random_state=42)

# Define the parameter grid for tuning
param_grid = {
    'max_depth': np.arange(2,11),         # ความลึกสูงสุดของต้นไม้
    'min_samples_split': np.arange(2,11),       # ขนาดขั้นต่ำของ sample ก่อนที่จะแบ่ง node
    'min_samples_leaf': np.arange(1,11), # ขนาดขั้นต่ำของ sample ใน leaf node
    'max_leaf_nodes' : [5,10,15,20]
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=dt_classifier,
                           param_grid=param_grid,
                           cv=5,                # 5-fold cross-validation
                           scoring='accuracy',  # ใช้คะแนนความแม่นยำ
                           n_jobs=-1)          # ใช้ทุก CPU core

# Fit the model to find the best parameters

grid_search.fit(x_train_selected70, y_train70)

# Display the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Use the best estimator for predictions
best_model = grid_search.best_estimator_
print(best_model)

Best Parameters: {'max_depth': np.int64(2), 'max_leaf_nodes': 5, 'min_samples_leaf': np.int64(1), 'min_samples_split': np.int64(2)}
Best Cross-Validation Score: 0.9144794816031215
DecisionTreeClassifier(max_depth=np.int64(2), max_leaf_nodes=5,
                       min_samples_leaf=np.int64(1),
                       min_samples_split=np.int64(2), random_state=42)


In [ ]:
# Train Decistion tree 70 :30 With Selection
dt_model = DecisionTreeClassifier(
    max_depth=9,                # ความลึกสูงสุดของต้นไม้
    max_leaf_nodes=20,           # จำนวน leaf nodes สูงสุด
    min_samples_leaf=1,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=2,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (70%)
dt_model.fit(x_train_selected70, y_train70)

# Make predictions on the test data (30%)
y_pred30 = dt_model.predict(x_test_selected30)

# แสดง Classification Report
accuracy = accuracy_score(y_test30, y_pred30)
precision = precision_score(y_test30, y_pred30)
recall = recall_score(y_test30, y_pred30)
f1 = f1_score(y_test30, y_pred30)
print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.9096
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Decision Tree with Feature Selection 80 : 20
# Define the Decision Tree model
dt_classifier = DecisionTreeClassifier(random_state=42)

# Define the parameter grid for tuning
param_grid = {
    'max_depth': np.arange(2,11),         # ความลึกสูงสุดของต้นไม้
    'min_samples_split': np.arange(2,11),       # ขนาดขั้นต่ำของ sample ก่อนที่จะแบ่ง node
    'min_samples_leaf': np.arange(1,11), # ขนาดขั้นต่ำของ sample ใน leaf node
    'max_leaf_nodes' : [5,10,15,20]
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=dt_classifier,
                           param_grid=param_grid,
                           cv=5,                # 5-fold cross-validation
                           scoring='accuracy',  # ใช้คะแนนความแม่นยำ
                           n_jobs=-1)          # ใช้ทุก CPU core

# Fit the model to find the best parameters

grid_search.fit(x_train_selected80, y_train80)

# Display the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Use the best estimator for predictions
best_model = grid_search.best_estimator_
print(best_model)

Best Parameters: {'max_depth': np.int64(2), 'max_leaf_nodes': 5, 'min_samples_leaf': np.int64(1), 'min_samples_split': np.int64(2)}
Best Cross-Validation Score: 0.9135583638172099
DecisionTreeClassifier(max_depth=np.int64(2), max_leaf_nodes=5,
                       min_samples_leaf=np.int64(1),
                       min_samples_split=np.int64(2), random_state=42)


In [ ]:
# Train Decistion tree 80 :20 With Selection
dt_model = DecisionTreeClassifier(
    max_depth=10,                # ความลึกสูงสุดของต้นไม้
    max_leaf_nodes=20,           # จำนวน leaf nodes สูงสุด
    min_samples_leaf=1,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=2,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (70%)
dt_model.fit(x_train_selected80, y_train80)

# Make predictions on the test data (30%)
y_pred20 = dt_model.predict(x_test_selected20)

# แสดง Classification Report
accuracy = accuracy_score(y_test20, y_pred20)
precision = precision_score(y_test20, y_pred20)
recall = recall_score(y_test20, y_pred20)
f1 = f1_score(y_test20, y_pred20)
print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.9366
Precision: 0.9281
Recall: 0.3126
F1 Score: 0.4676


# Decision Tree with SMOTE and Feature Selection

In [ ]:
# Decision Tree with SMOTE and Feature Selection 70 : 30

# Define the Decision Tree model
dt_classifier = DecisionTreeClassifier(random_state=42)

# Define the parameter grid for tuning
param_grid = {
    'max_depth': np.arange(2,11),         # ความลึกสูงสุดของต้นไม้
    'min_samples_split': np.arange(2,11),       # ขนาดขั้นต่ำของ sample ก่อนที่จะแบ่ง node
    'min_samples_leaf': np.arange(1,11), # ขนาดขั้นต่ำของ sample ใน leaf node
    'max_leaf_nodes' : [5,10,15,20]
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=dt_classifier,
                           param_grid=param_grid,
                           cv=5,                # 5-fold cross-validation
                           scoring='accuracy',  # ใช้คะแนนความแม่นยำ
                           n_jobs=-1)          # ใช้ทุก CPU core

# Fit the model to find the best parameters

grid_search.fit(x_train_smoteselected70, y_train_smote70)

# Display the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Use the best estimator for predictions
best_model = grid_search.best_estimator_
print(best_model)

Best Parameters: {'max_depth': 6, 'max_leaf_nodes': 20, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Cross-Validation Score: 0.8344415409670525
DecisionTreeClassifier(max_depth=6, max_leaf_nodes=20, random_state=42)


In [ ]:
# Train Decistion tree 70 :30 With SMOTE and Selection
dt_model = DecisionTreeClassifier(
    max_depth=6,                # ความลึกสูงสุดของต้นไม้
    max_leaf_nodes=20,           # จำนวน leaf nodes สูงสุด
    min_samples_leaf=1,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=2,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (70%)
dt_model.fit(x_train_smoteselected70, y_train_smote70)

# Make predictions on the test data (30%)
y_pred30 = dt_model.predict(x_test_smoteselected30)

# แสดง Classification Report
accuracy = accuracy_score(y_test_smote70, y_pred30)
precision = precision_score(y_test_smote70, y_pred30)
recall = recall_score(y_test_smote70, y_pred30)
f1 = f1_score(y_test_smote70, y_pred30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.8438
Precision: 0.8989
Recall: 0.7748
F1 Score: 0.8322


In [ ]:
# Decision Tree with SMOTE and Feature Selection 80 : 20

# Define the Decision Tree model
dt_classifier = DecisionTreeClassifier(random_state=42)

# Define the parameter grid for tuning
param_grid = {
    'max_depth': np.arange(2,11),         # ความลึกสูงสุดของต้นไม้
    'min_samples_split': np.arange(2,11),       # ขนาดขั้นต่ำของ sample ก่อนที่จะแบ่ง node
    'min_samples_leaf': np.arange(1,11), # ขนาดขั้นต่ำของ sample ใน leaf node
    'max_leaf_nodes' : [5,10,15,20]
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=dt_classifier,
                           param_grid=param_grid,
                           cv=5,                # 5-fold cross-validation
                           scoring='accuracy',  # ใช้คะแนนความแม่นยำ
                           n_jobs=-1)          # ใช้ทุก CPU core

# Fit the model to find the best parameters

grid_search.fit(x_train_smoteselected80, y_train_smote80)

# Display the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Use the best estimator for predictions
best_model = grid_search.best_estimator_
print(best_model)

Best Parameters: {'max_depth': 8, 'max_leaf_nodes': 20, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Cross-Validation Score: 0.8295991996633802
DecisionTreeClassifier(max_depth=8, max_leaf_nodes=20, random_state=42)


In [ ]:
# Train Decistion tree 80 :20 With SMOTE and Selection
dt_model = DecisionTreeClassifier(
    max_depth=8,                # ความลึกสูงสุดของต้นไม้
    max_leaf_nodes=20,           # จำนวน leaf nodes สูงสุด
    min_samples_leaf=1,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=2,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (80%)
dt_model.fit(x_train_smoteselected80, y_train_smote80)

# Make predictions on the test data (30%)
y_pred20 = dt_model.predict(x_test_smoteselected20)

# แสดง Classification Report
accuracy = accuracy_score(y_test_smote80, y_pred20)
precision = precision_score(y_test_smote80, y_pred20)
recall = recall_score(y_test_smote80, y_pred20)
f1 = f1_score(y_test_smote80, y_pred20)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.8500
Precision: 0.8941
Recall: 0.7941
F1 Score: 0.8411


# Random forest without SMOTE and Feature Selection

In [ ]:
# Random forest without SMOTE and Feature Selection 70 : 30

# สร้างโมเดล Random Forest
rf = RandomForestClassifier(random_state=42)

# กำหนดพารามิเตอร์สำหรับปรับแต่ง
param_grid = {
    'max_depth': [2, 5 ,7, 10],  # ความลึกของต้นไม้
    'min_samples_leaf': [1, 2, 4],   # จำนวนข้อมูลขั้นต่ำที่ต้องมีในใบ
    'min_samples_split': [2, 5, 10], # จำนวนข้อมูลขั้นต่ำที่ต้องมีเพื่อแยกโหนด
    'n_estimators': [100, 200, 300]   # จำนวนต้นไม้ในป่า
}

# ใช้ GridSearchCV สำหรับ Cross-Validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train70_scaled, y_train70)

# แสดงผลลัพธ์พารามิเตอร์ที่ดีที่สุด
print("Best Parameters:", grid_search.best_params_)


Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [ ]:
# สร้างโมเดล Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,            # จำนวนต้นไม้ในป่า
    max_depth=10,                # ความลึกสูงสุดของต้นไม้
    min_samples_leaf=1,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=2,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (70%)
rf_model.fit(x_train70_scaled, y_train70)

# Make predictions on the test data (30%)
y_pred30 = rf_model.predict(x_test30_scaled)

# แสดง Classification Report
accuracy = accuracy_score(y_test30, y_pred30)
precision = precision_score(y_test30, y_pred30)
recall = recall_score(y_test30, y_pred30)
f1 = f1_score(y_test30, y_pred30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))


Accuracy: 0.9209
Precision: 0.9769
Recall: 0.1281
F1 Score: 0.2265


In [ ]:
# Random forest without SMOTE and Feature Selection 80 : 20

# สร้างโมเดล Random Forest
rf = RandomForestClassifier(random_state=42)

# กำหนดพารามิเตอร์สำหรับปรับแต่ง
param_grid = {
    'max_depth': [2, 5 ,7, 10],  # ความลึกของต้นไม้
    'min_samples_leaf': [1, 2, 4],   # จำนวนข้อมูลขั้นต่ำที่ต้องมีในใบ
    'min_samples_split': [2, 5, 10], # จำนวนข้อมูลขั้นต่ำที่ต้องมีเพื่อแยกโหนด
    'n_estimators': [100, 200, 300]   # จำนวนต้นไม้ในป่า
}

# ใช้ GridSearchCV สำหรับ Cross-Validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train80_scaled, y_train80)

# แสดงผลลัพธ์พารามิเตอร์ที่ดีที่สุด
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}
Test Accuracy: 0.0891


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [ ]:

# สร้างโมเดล Random Forest   80 : 20
rf_model = RandomForestClassifier(
    n_estimators=300,            # จำนวนต้นไม้ในป่า
    max_depth=10,                # ความลึกสูงสุดของต้นไม้
    min_samples_leaf=2,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=5,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (80%)
rf_model.fit(x_train80_scaled, y_train80)

# Make predictions on the test data (20%)
y_pred20 = rf_model.predict(x_test20_scaled)

# แสดง Classification Report
accuracy = accuracy_score(y_test20, y_pred20)
precision = precision_score(y_test20, y_pred20)
recall = recall_score(y_test20, y_pred20)
f1 = f1_score(y_test20, y_pred20)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))


Accuracy: 0.9229
Precision: 0.9835
Recall: 0.1373
F1 Score: 0.2409


# Random Forest with SMOTE

In [ ]:
# Random Forest with SMOTE 70 : 30
# สร้างโมเดล Random Forest
rf = RandomForestClassifier(random_state=42)

# กำหนดพารามิเตอร์สำหรับปรับแต่ง
param_grid = {
    'max_depth': [2, 5 ,7, 10],  # ความลึกของต้นไม้
    'min_samples_leaf': [1, 2, 4],   # จำนวนข้อมูลขั้นต่ำที่ต้องมีในใบ
    'min_samples_split': [2, 5, 10], # จำนวนข้อมูลขั้นต่ำที่ต้องมีเพื่อแยกโหนด
    'n_estimators': [100, 200, 300]   # จำนวนต้นไม้ในป่า
}

# ใช้ GridSearchCV สำหรับ Cross-Validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train_smote70, y_train_smote70)

# แสดงผลลัพธ์พารามิเตอร์ที่ดีที่สุด
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}


In [ ]:
# สร้างโมเดล Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,            # จำนวนต้นไม้ในป่า
    max_depth=10,                # ความลึกสูงสุดของต้นไม้
    min_samples_leaf=1,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=10,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (70%)
rf_model.fit(x_train_smote70, y_train_smote70)

# Make predictions on the test data (30%)
y_pred30 = rf_model.predict(x_test_smote70)

# แสดง Classification Report
accuracy = accuracy_score(y_test_smote70, y_pred30)
precision = precision_score(y_test_smote70, y_pred30)
recall = recall_score(y_test_smote70, y_pred30)
f1 = f1_score(y_test_smote70, y_pred30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.8626
Precision: 0.8690
Recall: 0.8539
F1 Score: 0.8614


In [ ]:
# Random Forest with SMOTE 80 : 20

# สร้างโมเดล Random Forest
rf = RandomForestClassifier(random_state=42)

# กำหนดพารามิเตอร์สำหรับปรับแต่ง
param_grid = {
    'max_depth': [2, 5 ,7, 10],  # ความลึกของต้นไม้
    'min_samples_leaf': [1, 2, 4],   # จำนวนข้อมูลขั้นต่ำที่ต้องมีในใบ
    'min_samples_split': [2, 5, 10], # จำนวนข้อมูลขั้นต่ำที่ต้องมีเพื่อแยกโหนด
    'n_estimators': [100, 200, 300]   # จำนวนต้นไม้ในป่า
}

# ใช้ GridSearchCV สำหรับ Cross-Validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train_smote80, y_train_smote80)

# แสดงผลลัพธ์พารามิเตอร์ที่ดีที่สุด
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}


In [ ]:
# สร้างโมเดล Random Forest   80 : 20
rf_model = RandomForestClassifier(
    n_estimators=200,            # จำนวนต้นไม้ในป่า
    max_depth=10,                # ความลึกสูงสุดของต้นไม้
    min_samples_leaf=1,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=10,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (80%)
rf_model.fit(x_train_smote80, y_train_smote80)

# Make predictions on the test data (20%)
y_pred20 = rf_model.predict(x_test_smote80)

# แสดง Classification Report
accuracy = accuracy_score(y_test_smote80, y_pred20)
precision = precision_score(y_test_smote80, y_pred20)
recall = recall_score(y_test_smote80, y_pred20)
f1 = f1_score(y_test_smote80, y_pred20)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.8756
Precision: 0.8746
Recall: 0.8769
F1 Score: 0.8757


# Random Forest with Feature Selection

In [ ]:
# Random Forest with Feature Selection 70 : 30

# สร้างโมเดล Random Forest
rf = RandomForestClassifier(random_state=42)

# กำหนดพารามิเตอร์สำหรับปรับแต่ง
param_grid = {
    'max_depth': [2, 5 ,7, 10],  # ความลึกของต้นไม้
    'min_samples_leaf': [1, 2, 4],   # จำนวนข้อมูลขั้นต่ำที่ต้องมีในใบ
    'min_samples_split': [2, 5, 10], # จำนวนข้อมูลขั้นต่ำที่ต้องมีเพื่อแยกโหนด
    'n_estimators': [100, 200, 300]   # จำนวนต้นไม้ในป่า
}

# ใช้ GridSearchCV สำหรับ Cross-Validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train_selected70, y_train70)

# แสดงผลลัพธ์พารามิเตอร์ที่ดีที่สุด
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300}


In [ ]:
# สร้างโมเดล Random Forest
rf_model = RandomForestClassifier(
    n_estimators=300,            # จำนวนต้นไม้ในป่า
    max_depth=10,                # ความลึกสูงสุดของต้นไม้
    min_samples_leaf=4,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=2,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (70%)
rf_model.fit(x_train_selected70, y_train70)

# Make predictions on the test data (30%)
y_pred30 = rf_model.predict(x_test_selected30)

# แสดง Classification Report
accuracy = accuracy_score(y_test30, y_pred30)
precision = precision_score(y_test30, y_pred30)
recall = recall_score(y_test30, y_pred30)
f1 = f1_score(y_test30, y_pred30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.9358
Precision: 0.9401
Recall: 0.3093
F1 Score: 0.4655


In [ ]:
# Random Forest with Feature Selection 80 : 20
# สร้างโมเดล Random Forest
rf = RandomForestClassifier(random_state=42)

# กำหนดพารามิเตอร์สำหรับปรับแต่ง
param_grid = {
    'max_depth': [2, 5 ,7, 10],  # ความลึกของต้นไม้
    'min_samples_leaf': [1, 2, 4],   # จำนวนข้อมูลขั้นต่ำที่ต้องมีในใบ
    'min_samples_split': [2, 5, 10], # จำนวนข้อมูลขั้นต่ำที่ต้องมีเพื่อแยกโหนด
    'n_estimators': [100, 200, 300]   # จำนวนต้นไม้ในป่า
}

# ใช้ GridSearchCV สำหรับ Cross-Validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train_selected80, y_train80)

# แสดงผลลัพธ์พารามิเตอร์ที่ดีที่สุด
print("Best Parameters:", grid_search.best_params_)


Best Parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}


In [ ]:
# สร้างโมเดล Random Forest   80 : 20
rf_model = RandomForestClassifier(
    n_estimators=300,            # จำนวนต้นไม้ในป่า
    max_depth=10,                # ความลึกสูงสุดของต้นไม้
    min_samples_leaf=2,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=2,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (80%)
rf_model.fit(x_train_selected80, y_train80)

# Make predictions on the test data (20%)
y_pred20 = rf_model.predict(x_test_selected20)

# แสดง Classification Report
accuracy = accuracy_score(y_test20, y_pred20)
precision = precision_score(y_test20, y_pred20)
recall = recall_score(y_test20, y_pred20)
f1 = f1_score(y_test20, y_pred20)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.9367
Precision: 0.9197
Recall: 0.3172
F1 Score: 0.4717


# Random Forest with SMOTE and Feature Selection

In [ ]:
# Random Forest with SMOTE and Feature Selection 70 : 30
# สร้างโมเดล Random Forest
rf = RandomForestClassifier(random_state=42)

# กำหนดพารามิเตอร์สำหรับปรับแต่ง
param_grid = {
    'max_depth': [2, 5 ,7, 10],  # ความลึกของต้นไม้
    'min_samples_leaf': [1, 2, 4],   # จำนวนข้อมูลขั้นต่ำที่ต้องมีในใบ
    'min_samples_split': [2, 5, 10], # จำนวนข้อมูลขั้นต่ำที่ต้องมีเพื่อแยกโหนด
    'n_estimators': [100, 200, 300]   # จำนวนต้นไม้ในป่า
}

# ใช้ GridSearchCV สำหรับ Cross-Validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train_smoteselected70, y_train_smote70)

# แสดงผลลัพธ์พารามิเตอร์ที่ดีที่สุด
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}


In [ ]:
# สร้างโมเดล Random Forest
rf_model = RandomForestClassifier(
    n_estimators=200,            # จำนวนต้นไม้ในป่า
    max_depth=10,                # ความลึกสูงสุดของต้นไม้
    min_samples_leaf=1,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=10,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (70%)
rf_model.fit(x_train_smoteselected70,  y_train_smote70)

# Make predictions on the test data (30%)
y_pred30 = rf_model.predict(x_test_smoteselected30)


# แสดง Classification Report
accuracy = accuracy_score(y_test_smote70, y_pred30)
precision = precision_score(y_test_smote70, y_pred30)
recall = recall_score(y_test_smote70, y_pred30)
f1 = f1_score(y_test_smote70, y_pred30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.8846
Precision: 0.9219
Recall: 0.8403
F1 Score: 0.8792


In [ ]:
# Random Forest with SMOTE and Feature Selection 80 : 20
# สร้างโมเดล Random Forest
rf = RandomForestClassifier(random_state=42)

# กำหนดพารามิเตอร์สำหรับปรับแต่ง
param_grid = {
    'max_depth': [2, 5 ,7, 10],  # ความลึกของต้นไม้
    'min_samples_leaf': [1, 2, 4],   # จำนวนข้อมูลขั้นต่ำที่ต้องมีในใบ
    'min_samples_split': [2, 5, 10], # จำนวนข้อมูลขั้นต่ำที่ต้องมีเพื่อแยกโหนด
    'n_estimators': [100, 200, 300]   # จำนวนต้นไม้ในป่า
}

# ใช้ GridSearchCV สำหรับ Cross-Validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train_smoteselected80, y_train_smote80)

# แสดงผลลัพธ์พารามิเตอร์ที่ดีที่สุด
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}


In [ ]:
# สร้างโมเดล Random Forest
rf_model = RandomForestClassifier(
    n_estimators=300,            # จำนวนต้นไม้ในป่า
    max_depth=10,                # ความลึกสูงสุดของต้นไม้
    min_samples_leaf=1,          # จำนวนขั้นต่ำของข้อมูลใน leaf node
    min_samples_split=10,         # จำนวนขั้นต่ำของข้อมูลก่อนแยกโหนด
    random_state=42              # สำหรับ reproducibility
)

# Train the model on the training data (80%)
rf_model.fit(x_train_smoteselected80,  y_train_smote80)

# Make predictions on the test data (20%)
y_pred20 = rf_model.predict(x_test_smoteselected20)


# แสดง Classification Report
accuracy = accuracy_score(y_test_smote80, y_pred20)
precision = precision_score(y_test_smote80, y_pred20)  # ใช้ average='weighted' สำหรับ multiclass
recall = recall_score(y_test_smote80, y_pred20)
f1 = f1_score(y_test_smote80, y_pred20)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.8920
Precision: 0.9280
Recall: 0.8500
F1 Score: 0.8873


# XGBoost without SMOTE and Feature Selection

In [ ]:
# XGBoost without SMOTE and Feature Selection 70 : 30

#x_train70['awards_won?'] = x_train70['awards_won?'].cat.codes
#x_test30['awards_won?'] = x_test30['awards_won?'].cat.codes
# Define model
model = xgb.XGBClassifier(eval_metric='logloss', random_state=42)


# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': np.arange(3, 11),
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
}

# Use GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',  # Use an appropriate scoring metric for classification
    cv=5,
    verbose=1
)

# Fit model
grid_search.fit(x_train70, y_train70)
# Output results
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

Fitting 5 folds for each of 160 candidates, totalling 800 fits
Best Parameters: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 500}
Best Cross-Validation Accuracy: 0.942868789322751


In [ ]:
# Training Model 70 : 30

model  =xgb.XGBClassifier(n_estimators=500,
                          max_depth=4,
                          learning_rate=0.1)

# Train the model
model.fit(x_train70, y_train70)

# Predict
y_pred30 = model.predict(x_test30)

# แสดง Classification Report
accuracy = accuracy_score(y_test30, y_pred30)
precision = precision_score(y_test30, y_pred30)
recall = recall_score(y_test30, y_pred30)
f1 = f1_score(y_test30, y_pred30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Accuracy: 0.9384
Precision: 0.9449
Recall: 0.3381
F1 Score: 0.4980


In [ ]:
# XGBoost without SMOTE and Feature Selection 80 : 20

# Define model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300,400,500],
    'max_depth':np.arange(3,11),
    'learning_rate': [0.01, 0.1, 0.2,0.3],
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=2,
    n_jobs=-1
)

# Fit model
grid_search.fit(x_train80, y_train80)

# Output results
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)


Fitting 5 folds for each of 160 candidates, totalling 800 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:40:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 200}
Best Cross-Validation Accuracy: 0.9421238621236245


In [ ]:
# Training Model 80 : 20

model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=200,       # จำนวนต้นไม้
    max_depth=3,            # ความลึกสูงสุดของต้นไม้
    learning_rate=0.3,      # อัตราการเรียนรู้
)

#x_train80['awards_won?'] = x_train80['awards_won?'].cat.codes
# Train the model
model.fit(x_train80, y_train80)


# Predict
y_pred20 = model.predict(x_test20)

# แสดง Classification Report
accuracy = accuracy_score(y_test20, y_pred20)
precision = precision_score(y_test20, y_pred20)
recall = recall_score(y_test20, y_pred20)
f1 = f1_score(y_test20, y_pred20)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:54:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.9408
Precision: 0.9619
Recall: 0.3495
F1 Score: 0.5127


# XGBoost with SMOTE

In [ ]:
# XGBoost with SMOTE 70 : 30

# Define model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300,400,500],
    'max_depth':np.arange(3,11),
    'learning_rate': [0.01, 0.1, 0.2,0.3],
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=2,
    n_jobs=-1
)

# Fit model
grid_search.fit(x_train_smote70, y_train_smote70)

# Output results
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:22:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'learning_rate': 0.1, 'max_depth': np.int64(10), 'n_estimators': 500}
Best Cross-Validation Accuracy: 0.9582982178334666


In [ ]:
# Training Model 70 : 30  with SMOTE

model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=500,       # จำนวนต้นไม้
    max_depth=10,            # ความลึกสูงสุดของต้นไม้
    learning_rate=0.1,      # อัตราการเรียนรู้
)

#x_train_smote70['awards_won?'] =x_train_smote70['awards_won?'].cat.codes
# Train the model
model.fit(x_train_smote70, y_train_smote70)


# Predict
y_pred30 = model.predict(x_test_smote70)

# แสดง Classification Report
accuracy = accuracy_score( y_test_smote70,y_pred30)
precision = precision_score( y_test_smote70, y_pred30)
recall = recall_score( y_test_smote70, y_pred30)
f1 = f1_score( y_test_smote70, y_pred30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:53:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.9522
Precision: 0.9891
Recall: 0.9145
F1 Score: 0.9503


In [ ]:
# XGBoost with SMOTE 80 : 20

# Define model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300,400,500],
    'max_depth':np.arange(3,11),
    'learning_rate': [0.01, 0.1, 0.2,0.3],
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=2,
    n_jobs=-1
)

# Fit model
grid_search.fit(x_train_smote80, y_train_smote80)

# Output results
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)


Fitting 5 folds for each of 160 candidates, totalling 800 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:19:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'learning_rate': 0.1, 'max_depth': np.int64(10), 'n_estimators': 500}
Best Cross-Validation Accuracy: 0.9580620735270973


In [ ]:
# Training Model 80 : 20

model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=500,       # จำนวนต้นไม้
    max_depth=10,            # ความลึกสูงสุดของต้นไม้
    learning_rate=0.1,      # อัตราการเรียนรู้
)

#x_train_smote80['awards_won?'] =x_train_smote80['awards_won?'].cat.codes
# Train the model
model.fit(x_train_smote80, y_train_smote80)


# Predict
y_pred20 = model.predict(x_test_smote80)

# แสดง Classification Report
accuracy = accuracy_score( y_test_smote80,y_pred20)
precision = precision_score( y_test_smote80, y_pred20)
recall = recall_score( y_test_smote80, y_pred20)
f1 = f1_score( y_test_smote80, y_pred20)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:51:16] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.9528
Precision: 0.9875
Recall: 0.9173
F1 Score: 0.9511


# XGBoost with Feature Selection

In [ ]:
# XGBoost with Feature Selection 70 : 30

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300,400,500],
    'max_depth':np.arange(3,11),
    'learning_rate': [0.01, 0.1, 0.2,0.3],
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=2,
    n_jobs=-1
)

# Fit model
grid_search.fit(x_train_selected70, y_train70)

# Output results
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)


Fitting 5 folds for each of 160 candidates, totalling 800 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:59:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'learning_rate': 0.01, 'max_depth': np.int64(3), 'n_estimators': 100}
Best Cross-Validation Accuracy: 0.9144794816031215


In [ ]:
# Training Model 70 : 30  with Feature Selection

model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=100,       # จำนวนต้นไม้
    max_depth=3,            # ความลึกสูงสุดของต้นไม้
    learning_rate=0.01,      # อัตราการเรียนรู้
)


# Train the model
model.fit(x_train_selected70, y_train70)


# Make predictions on the test data (30%)
y_pred30 = model.predict(x_test_selected30)

# แสดง Classification Report
accuracy = accuracy_score(y_test30, y_pred30)
precision = precision_score(y_test30, y_pred30)
recall = recall_score(y_test30, y_pred30)
f1 = f1_score(y_test30, y_pred30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:01:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.9096
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# XGBoost with Feature Selection 80 : 20

# Define model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300,400,500],
    'max_depth':np.arange(3,11),
    'learning_rate': [0.01, 0.1, 0.2,0.3],
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=2,
    n_jobs=-1
)

# Fit model
grid_search.fit(x_train_selected80, y_train80)

# Output results
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:14:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300}
Best Cross-Validation Accuracy: 0.9393751807544163


In [ ]:
# Training Model 80 : 20

model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=100,       # จำนวนต้นไม้
    max_depth=3,            # ความลึกสูงสุดของต้นไม้
    learning_rate=0.01,      # อัตราการเรียนรู้
)

# Train the model
model.fit(x_train_selected80, y_train80)


# Make predictions on the test data (30%)
y_pred30 = model.predict(x_test_selected20)

# แสดง Classification Report
accuracy = accuracy_score(y_test20, y_pred30)
precision = precision_score(y_test20, y_pred30)
recall = recall_score(y_test20, y_pred30)
f1 = f1_score(y_test20, y_pred30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:02:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.9109
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# XGBoost with SMOTE and Feature Selection

In [ ]:
# XGBoost with SMOTE and Feature Selection 70 : 30

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300,400,500],
    'max_depth':np.arange(3,11),
    'learning_rate': [0.01, 0.1, 0.2,0.3],
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=2,
    n_jobs=-1
)

# Fit model
grid_search.fit(x_train_smoteselected70,  y_train_smote70)

# Output results
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)


Fitting 5 folds for each of 160 candidates, totalling 800 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:17:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'learning_rate': 0.2, 'max_depth': 4, 'n_estimators': 400}
Best Cross-Validation Accuracy: 0.91950074317598


In [ ]:
# XGBoost with SMOTE and Feature Selection 70 : 30
# Training Model 70 : 30  with Feature Selection

model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=400,       # จำนวนต้นไม้
    max_depth=4,            # ความลึกสูงสุดของต้นไม้
    learning_rate=0.2,      # อัตราการเรียนรู้
)

# Train the model on the training data (70%)
model.fit(x_train_smoteselected70,  y_train_smote70)

# Make predictions on the test data (30%)
y_pred30 = model.predict(x_test_smoteselected30)


# แสดง Classification Report
accuracy = accuracy_score(y_test_smote70, y_pred30)
precision = precision_score(y_test_smote70, y_pred30)
recall = recall_score(y_test_smote70, y_pred30)
f1 = f1_score(y_test_smote70, y_pred30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:25:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.9243
Precision: 0.9799
Recall: 0.8663
F1 Score: 0.9196


In [ ]:
# XGBoost with SMOTE and Feature Selection 80 : 20

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300,400,500],
    'max_depth':np.arange(3,11),
    'learning_rate': [0.01, 0.1, 0.2,0.3],
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=2,
    n_jobs=-1
)

# Fit model
grid_search.fit(x_train_smoteselected80,  y_train_smote80)

# Output results
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)


Fitting 5 folds for each of 160 candidates, totalling 800 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:21:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 500}
Best Cross-Validation Accuracy: 0.9166426422375229


In [ ]:
# XGBoost with SMOTE and Feature Selection 80 : 20

model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=500,       # จำนวนต้นไม้
    max_depth=4,            # ความลึกสูงสุดของต้นไม้
    learning_rate=0.3,      # อัตราการเรียนรู้
)

# Train the model on the training data (70%)
model.fit(x_train_smoteselected80,  y_train_smote80)

# Make predictions on the test data (30%)
y_pred30 = model.predict(x_test_smoteselected20)


# แสดง Classification Report
accuracy = accuracy_score(y_test_smote80, y_pred30)
precision = precision_score(y_test_smote80, y_pred30)
recall = recall_score(y_test_smote80, y_pred30)
f1 = f1_score(y_test_smote80, y_pred30)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:23:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.9239
Precision: 0.9771
Recall: 0.8680
F1 Score: 0.9194
